In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.CDSSM.get_default_preprocessor(
    ngram_size = 3
)

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7970.11it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4029.12it/s]
Building Vocabulary from a datapack.: 100%|██████████| 2082963/2082963 [00:00<00:00, 2585708.82it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8438.27it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4307.49it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 773018.24it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8153.75it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [6]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x7fdcf7104a58>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fdcf7104b38>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fdd6f119390>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fdcf4169dd8>,
 'ngram_vocab_size': 9654}

In [7]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False,
    callbacks=[triletter_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    sort=False,
    callbacks=[triletter_callback]
)

In [8]:
padding_callback = mz.models.CDSSM.get_default_padding_callback(
    with_ngram=True,
    fixed_ngram_length=preprocessor.context['ngram_vocab_size']
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(9654, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(9654, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): T

In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-102 Loss-0.732]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3381 - normalized_discounted_cumulative_gain@5(0.0): 0.4153 - mean_average_precision(0.0): 0.3812



[Iter-204 Loss-0.583]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3271 - normalized_discounted_cumulative_gain@5(0.0): 0.4219 - mean_average_precision(0.0): 0.3834



[Iter-306 Loss-0.466]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3289 - normalized_discounted_cumulative_gain@5(0.0): 0.4134 - mean_average_precision(0.0): 0.3825



[Iter-408 Loss-0.370]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3299 - normalized_discounted_cumulative_gain@5(0.0): 0.418 - mean_average_precision(0.0): 0.3862



[Iter-510 Loss-0.319]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3396 - normalized_discounted_cumulative_gain@5(0.0): 0.4149 - mean_average_precision(0.0): 0.3856



[Iter-612 Loss-0.272]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3175 - normalized_discounted_cumulative_gain@5(0.0): 0.4051 - mean_average_precision(0.0): 0.3711



[Iter-714 Loss-0.253]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.336 - normalized_discounted_cumulative_gain@5(0.0): 0.4312 - mean_average_precision(0.0): 0.3893



[Iter-816 Loss-0.231]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3274 - normalized_discounted_cumulative_gain@5(0.0): 0.4114 - mean_average_precision(0.0): 0.3663



[Iter-918 Loss-0.194]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3376 - normalized_discounted_cumulative_gain@5(0.0): 0.4275 - mean_average_precision(0.0): 0.3854



[Iter-1020 Loss-0.166]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3268 - normalized_discounted_cumulative_gain@5(0.0): 0.4244 - mean_average_precision(0.0): 0.3827

Cost time: 322.57340145111084s
